## Paquets i dades originals ## 

In [318]:
import json
import pandas as pd
from datetime import datetime
from listas import disfagia_list, PA_list, P_list, Main_respiratory_infections_list, LRTI_list, COPD_exacerbations_list, Pulmonary_fibrosis_fibrotorax_list
from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtenir_ultim_resultat, obtenir_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

taula = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


Prèviament, he tractat les dades creant noves columnes que classifiquen millor tota la informació disponible

In [319]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,pes més antic,pes més nou,data pes més antic,data primer MECVV,pes coincident primer MECVV,pèrdua pes entre ingressos,perdua pes total,data més antiga pneumònia,dies entre primer ICD pneumìnia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


Distribuiré les dades en 3 grups (més un grup que conté la totalitat de dades):
- **AMB_PA**: aquest grup està compost per pacients que contenen el codi ICD de pneumònia aspirativa al seu historial.
- **AMB_PA_MECVV**: aquest grup conté els pacients que compleixen les següents característiques: 
    - Han donat positiu al MECVV (és a dir, que tenen disfàgia i una alteració en la seguretat o en l'eficàcia)
    - Han sigut diagnosticats amb el codi ICD de pneumònia
    - Han passat menys de 30 dies entre el diagnòstic amb l'ICD de pneumònia i el resultat positiu al MECVV
- **SENSE_PA**: aquest grup està format per pacients que contenen el codi ICD de pneumònia o bé:
    - Contenen el codi ICD de pneumònia
    - Han donat positiu al MECVV 
    - Han passat més de 30 dies entre el diagnòstic amb l'ICD de pneumònia i  el resultat positiu del MECVV 
- **TOTAL**: aquest grup conté tots els pacients de l'estudi *#TODO: hacer descriptiva de este grupo*

# AMB_PA 

In [320]:
AMB_PA = taula[taula["PA_diagnosticada"] == 1.0]
AMB_PA.head()

KeyError: 'PA_diagnosticada'

# AMB_PA_MECVV

In [321]:
AMB_PA_MECVV = taula[
    (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] < 30)] 
AMB_PA_MECVV.head()

KeyError: 'Dies entre primer ICD pneumònia i primer MECVV positiu'

# SENSE_PA

In [322]:
SENSE_PA = taula[
           (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] > 30) & 
           (taula['P_diagnosticada'] == 1.0)] 
SENSE_PA.head()

KeyError: 'Dies entre primer ICD pneumònia i primer MECVV positiu'

# TOTAL

In [323]:
TOTAL = taula
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,pes més antic,pes més nou,data pes més antic,data primer MECVV,pes coincident primer MECVV,pèrdua pes entre ingressos,perdua pes total,data més antiga pneumònia,dies entre primer ICD pneumìnia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


Ara que tinc totes les dades classificades en aquests grups, faré una descriptiva. També compararé els diferents grups amb el pvalor mitjançant T-test, test de Mann-Whitney o test Xi-quadrat per mirar la seva distribució, la qual esperaré que sigui similar entre el grup AMB_PA i AMB_PA_MECVV, i diferent entre SENSE_PA i els altres 2 grups. Els criteris que seguiré per utilitzar un test o un altre és, primer, fer un test de Shapiro-Wilk per comprovar si segueixen una distribució normal, si és així, usaré el T-test, si no, utilitzaré el de Mann-Whitney. Finalment, si la columna conté valors categòrics com és el cas del sexe (F/M), faré servir el test Xi-quadrat.


## Nombre de pacients

In [324]:
num_AMB_PA = len(AMB_PA)
num_AMB_PA_MECVV= len(AMB_PA_MECVV)
num_SENSE_PA = len(SENSE_PA)
num_TOTAL = len(TOTAL)

print(num_AMB_PA, num_AMB_PA_MECVV, num_SENSE_PA, num_TOTAL)

17 8 8 50


#### Pvalor

##### Normalitat

In [325]:
stat, p_value = shapiro(range(num_AMB_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9621
Valor p = 0.6704
El nombre de files segueix una distribució normal.


In [326]:
stat, p_value = shapiro(range(num_AMB_PA_MECVV))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9749
Valor p = 0.9332
El nombre de files segueix una distribució normal.


In [327]:
stat, p_value = shapiro(range(num_SENSE_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9749
Valor p = 0.9332
El nombre de files segueix una distribució normal.


In [328]:
stat, p_value = shapiro(range(num_TOTAL))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9556
Valor p = 0.0581
El nombre de files segueix una distribució normal.


In [329]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA.index, AMB_PA_MECVV.index, equal_var=False)
print(f"P-valor entre AMB_PA i AMB_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [330]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA.index, SENSE_PA.index, equal_var=False)
print(f"P-valor entre AMB_PA i SENSE_PA: {p_val_13:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i SENSE_PA: 0.0338
No existeixen diferències significatives entre els 2 grups.


In [331]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV.index, SENSE_PA.index, equal_var=False)
print(f"P-valor entre AMB_PA_MECVV i SENSE_PA: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [332]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA.index, AMB_PA_MECVV.index, equal_var=False)
print(f"P-valor entre AMB_PA i AMB_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [333]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA.index, TOTAL.index, equal_var=False)
print(f"P-valor entre AMB_PA i TOTAL: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i TOTAL: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [334]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV.index, TOTAL.index, equal_var=False)
print(f"P-valor entre AMB_PA_MECVV i TOTAL: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA_MECVV i TOTAL: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [335]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA.index, TOTAL.index, equal_var=False)
print(f"P-valor entre SENSE_PA i TOTAL: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre SENSE_PA i TOTAL: 0.7033
No existeixen diferències significatives entre els 2 grups.


## Edat

In [336]:
edad_AMB_PA = AMB_PA['edat'].mean()
desviacion_estandar_edad = AMB_PA['edat'].std()

resultado = f"{edad_AMB_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)


Mitjana ± Desviació Estàndard de l'edat: 87.06 ± 8.99


In [337]:
edad_AMB_PA_MECVV = AMB_PA_MECVV['edat'].mean()
desviacion_estandar_edad = AMB_PA_MECVV['edat'].std()

resultado = f"{edad_AMB_PA_MECVV:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 91.50 ± 6.82


In [338]:
edad_SENSE_PA = SENSE_PA['edat'].mean()
desviacion_estandar_edad = SENSE_PA['edat'].std()

resultado = f"{edad_SENSE_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 86.25 ± 6.90


In [339]:
edad_TOTAL = TOTAL['edat'].mean()
desviacion_estandar_TOTAL = TOTAL['edat'].std()

resultado = f"{edad_SENSE_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 86.25 ± 6.90


#### Pvalor

##### Normalitat 

In [340]:
stat, p_value = shapiro(AMB_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9410
Valor p = 0.3297
El nombre de files segueix una distribució normal.


In [341]:
stat, p_value = shapiro(AMB_PA_MECVV['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8304
Valor p = 0.0600
El nombre de files segueix una distribució normal.


In [342]:
stat, p_value = shapiro(SENSE_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8318
Valor p = 0.0619
El nombre de files segueix una distribució normal.


In [343]:
stat, p_value = shapiro(TOTAL['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8491
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [344]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['edat'], AMB_PA_MECVV['edat'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.18896133816771837


In [345]:
stat, p_value_13 = mannwhitneyu(AMB_PA['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.9534289321662349


In [346]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.37026415517950617


In [347]:
stat, p_value_14 = mannwhitneyu(AMB_PA['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

P-valor entre AMB_PA i TOTAL 0.20417514607442822


In [348]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL 0.05636751107260842


In [349]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

P-valor entre SENSE_PA i TOTAL 0.4769108663506575


## Sexe

In [350]:
sexe_AMB_PA = AMB_PA['sexe'].value_counts()
sexe_AMB_PA_MECVV = AMB_PA_MECVV['sexe'].value_counts()
sexe_SENSE_PA = SENSE_PA['sexe'].value_counts()
sexe_TOTAL = TOTAL['sexe'].value_counts()

In [351]:
if 'F' in sexe_AMB_PA:
    percent_female1 = (sexe_AMB_PA['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA")

Percentatge de sexe femení en AMB_PA: 47.06%


In [352]:
if 'F' in sexe_AMB_PA_MECVV:
    percent_female2 = (sexe_AMB_PA_MECVV['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA_MECVV: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA_MECVV")

Percentatge de sexe femení en AMB_PA_MECVV: 47.06%


In [353]:
if 'F' in sexe_SENSE_PA:
    percent_female3 = (sexe_SENSE_PA['F'] / len(SENSE_PA)) * 100
    print(f"Percentatge de sexe femení en SENSE_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en SENSE_PA")


Percentatge de sexe femení en SENSE_PA: 47.06%


In [354]:
if 'F' in sexe_TOTAL:
    percent_female3 = (sexe_TOTAL['F'] / len(TOTAL)) * 100
    print(f"Percentatge de sexe femení en TOTAL: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en TOTAL")

Percentatge de sexe femení en TOTAL: 47.06%


#### Pvalor

##### Normalitat

Al ser una variable categòrica no es pot comprovar la seva normalitat. Realitzaré directament el test Xi-quadrat

In [355]:
contingency_table_AMB_PA_AMB_PA_MECVV = pd.crosstab(index=AMB_PA['sexe'], columns=AMB_PA_MECVV['sexe'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_AMB_PA_AMB_PA_MECVV)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.


In [356]:
contingency_table_AMB_PA_SENSE_PA = pd.crosstab(index=AMB_PA['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.


In [357]:
contingency_table_AMB_PA_MECVV_SENSE_PA = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_MECVV_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [358]:
contingency_table_14 = pd.crosstab(index=AMB_PA['sexe'], columns=TOTAL['sexe'])

chi2_AMB_PA_TOTAL, p_value_AMB_PA_TOTAL, dof_AMB_PA_TOTAL, expected_AMB_PA_TOTAL = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_AMB_PA_TOTAL)
print("P-valor entre AMB_PA i TOTAL:", p_value_AMB_PA_TOTAL)

if p_value_AMB_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 13.223042052469136
P-valor entre AMB_PA i TOTAL: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.


In [359]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=TOTAL['sexe'])

chi2_AMB_PA_MECVV_TOTAL, p_value_AMB_PA_MECVV_TOTAL, dof_AMB_PA_MECVV_TOTAL, expected_AMB_PA_MECVV_TOTAL = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_AMB_PA_MECVV_TOTAL)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_AMB_PA_MECVV_TOTAL)

if p_value_AMB_PA_MECVV_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.


In [360]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['sexe'], columns=TOTAL['sexe'])

chi2_SENSE_PA_TOTAL, p_value_SENSE_PA_TOTAL, dof_SENSE_PA_TOTAL, expected_SENSE_PA_TOTAL = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_SENSE_PA_TOTAL)
print("P-valor entre SENSE_PA i TOTAL:", p_value_SENSE_PA_TOTAL)

if p_value_SENSE_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.5
P-valor entre SENSE_PA i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.


## Ingressos (admissions)

In [361]:
promedio_ingresos = AMB_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = AMB_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.59 ± 3.37


In [362]:
promedio_ingresos = AMB_PA_MECVV['Num_ingresos'].mean()
desviacion_estandar_ingresos = AMB_PA_MECVV['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.88 ± 1.64


In [363]:
promedio_ingresos = SENSE_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = SENSE_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 8.75 ± 3.85


In [364]:
promedio_ingresos = TOTAL['Num_ingresos'].mean()
desviacion_estandar_ingresos = TOTAL['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 6.68 ± 5.08


#### Pvalor

##### Normalitat

In [365]:
stat, p_value = shapiro(AMB_PA['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8008
Valor p = 0.0021
El nombre de files no segueix una distribució normal.


In [366]:
stat, p_value = shapiro(AMB_PA_MECVV['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9122
Valor p = 0.3695
El nombre de files segueix una distribució normal.


In [367]:
stat, p_value = shapiro(SENSE_PA['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9498
Valor p = 0.7091
El nombre de files segueix una distribució normal.


In [368]:
stat, p_value = shapiro(TOTAL['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7256
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [369]:
stat, p_value = shapiro(TOTAL['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7256
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [370]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Num_ingresos'], AMB_PA_MECVV['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.27403132109149697


In [371]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Num_ingresos'], SENSE_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.042821773602008555


In [372]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Num_ingresos'], SENSE_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.18434809692120802


In [373]:
stat, p_value_14 = mannwhitneyu(AMB_PA['Num_ingresos'], TOTAL['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

P-valor entre AMB_PA i TOTAL 0.4941763107233279


In [374]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Num_ingresos'], TOTAL['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL 0.6902658476564807


In [375]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['Num_ingresos'], TOTAL['Num_ingresos'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

P-valor entre SENSE_PA i TOTAL 0.06577367478634001


## Disfàgia

In [376]:
disf_AMB_PA= (AMB_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA:.2f}%")

Percentatge de disfàgia: 47.06%


In [377]:
disf_AMB_PA_MECVV = (AMB_PA_MECVV['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA_MECVV:.2f}%")

Percentatge de disfàgia: 50.00%


In [378]:
disf_SENSE_PA= (SENSE_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_SENSE_PA:.2f}%")

Percentatge de disfàgia: 25.00%


In [379]:
disf_TOTAL= (TOTAL['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_TOTAL:.2f}%")

KeyError: 'DO_diagnosticada'

#### Pvalor

##### Normalitat

In [380]:
stat, p_value = shapiro(AMB_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [381]:
stat, p_value = shapiro(AMB_PA_MECVV['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [382]:
stat, p_value = shapiro(SENSE_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [383]:
stat, p_value = shapiro(TOTAL['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

KeyError: 'DO_diagnosticada'

In [384]:
contingency_table_12 = pd.crosstab(index=AMB_PA['DO_diagnosticada'], columns=AMB_PA_MECVV['DO_diagnosticada'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.


In [385]:
contingency_table_13 = pd.crosstab(index=AMB_PA['DO_diagnosticada'], columns=SENSE_PA['DO_diagnosticada'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.


In [386]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['DO_diagnosticada'], columns=SENSE_PA['DO_diagnosticada'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [387]:
contingency_table_14 = pd.crosstab(index=AMB_PA['DO_diagnosticada'], columns=TOTAL['DO_diagnosticada'])

chi2_AMB_PA_TOTAL, p_value_AMB_PA_TOTAL, dof_AMB_PA_TOTAL, expected_AMB_PA_TOTAL = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_AMB_PA_TOTAL)
print("P-valor entre AMB_PA i TOTAL:", p_value_AMB_PA_TOTAL)

if p_value_AMB_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.")

KeyError: 'DO_diagnosticada'

In [388]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['DO_diagnosticada'], columns=TOTAL['DO_diagnosticada'])

chi2_AMB_PA_MECVV_TOTAL, p_value_AMB_PA_MECVV_TOTAL, dof_AMB_PA_MECVV_TOTAL, expected_AMB_PA_MECVV_TOTAL = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_AMB_PA_MECVV_TOTAL)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_AMB_PA_MECVV_TOTAL)

if p_value_AMB_PA_MECVV_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.")

KeyError: 'DO_diagnosticada'

In [389]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['DO_diagnosticada'], columns=TOTAL['DO_diagnosticada'])

chi2_SENSE_PA_TOTAL, p_value_SENSE_PA_TOTAL, dof_SENSE_PA_TOTAL, expected_SENSE_PA_TOTAL = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_SENSE_PA_TOTAL)
print("P-valor entre SENSE_PA i TOTAL:", p_value_SENSE_PA_TOTAL)

if p_value_SENSE_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.")

KeyError: 'DO_diagnosticada'

## Index de Barthel

### Promedio

In [390]:
promedio_barthel = AMB_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = AMB_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 22.65 ± 24.63


In [391]:
promedio_barthel = AMB_PA_MECVV['Barthel_resultados'].mean()
desviacion_estandar_barthel = AMB_PA_MECVV['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 13.75 ± 18.27


In [392]:
promedio_barthel = SENSE_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = SENSE_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 53.12 ± 21.20


#### Pvalor

##### Normalitat

In [393]:
stat, p_value = shapiro(AMB_PA['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8517
Valor p = 0.0114
El nombre de files no segueix una distribució normal.


In [394]:
stat, p_value = shapiro(AMB_PA_MECVV['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8000
Valor p = 0.0286
El nombre de files no segueix una distribució normal.


In [395]:
stat, p_value = shapiro(SENSE_PA['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9475
Valor p = 0.6859
El nombre de files segueix una distribució normal.


In [396]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'], AMB_PA_MECVV['Barthel_resultados'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.47528505928431597


In [397]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'], SENSE_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.00906817282777616


In [398]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'], SENSE_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.006056441109782507


### Valors de dependència

##### AMB_PA

In [399]:
indepe_AMB_PA = (AMB_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA: {indepe_AMB_PA:.2f}%")

Percentatge de indepe_AMB_PA: 0.00%


In [400]:
mod_AMB_PA = (AMB_PA['Barthel_resultados'].between(61, 95)).mean() * 100
print(f"Percentatge de mod_AMB_PA: {mod_AMB_PA:.2f}%")

Percentatge de mod_AMB_PA: 5.88%


In [401]:
sev_AMB_PA = (AMB_PA['Barthel_resultados'].between(21, 60)).mean() * 100
print(f"Percentatge de sev_AMB_PA: {sev_AMB_PA:.2f}%")

Percentatge de sev_AMB_PA: 41.18%


In [402]:
total_AMB_PA = (AMB_PA['Barthel_resultados'] <= 20.0 ).mean() * 100
print(f"Percentatge de total_AMB_PA: {total_AMB_PA:.2f}%")

Percentatge de total_AMB_PA: 52.94%


##### AMB_PA_MECVV

In [403]:
indepe_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA_MECVV: {indepe_AMB_PA_MECVV:.2f}%")

Percentatge de indepe_AMB_PA_MECVV: 0.00%


In [404]:
mod_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Percentatge de mod_AMB_PA_MECVV: {mod_AMB_PA_MECVV:.2f}%")

Percentatge de mod_AMB_PA_MECVV: 0.00%


In [405]:
sev_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Percentatge de sev_AMB_PA_MECVV: {sev_AMB_PA_MECVV:.2f}%")

Percentatge de sev_AMB_PA_MECVV: 25.00%


In [406]:
total_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'] <= 20.0).mean() * 100
print(f"Percentatge de total_AMB_PA_MECVV: {total_AMB_PA_MECVV:.2f}%")

Percentatge de total_AMB_PA_MECVV: 75.00%


##### SENSE_PA

In [407]:
indepe_SENSE_PA = (SENSE_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_SENSE_PA: {indepe_SENSE_PA:.2f}%")

Percentatge de indepe_SENSE_PA: 0.00%


In [408]:
mod_SENSE_PA = (SENSE_PA['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Percentatge de mod_SENSE_PA: {mod_SENSE_PA:.2f}%")

Percentatge de mod_SENSE_PA: 37.50%


In [409]:
sev_SENSE_PA = (SENSE_PA['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Percentatge de sev_SENSE_PA: {sev_SENSE_PA:.2f}%")

Percentatge de sev_SENSE_PA: 62.50%


In [410]:
total_SENSE_PA = (SENSE_PA['Barthel_resultados'] <= 20).mean() * 100
print(f"Percentatge de total_SENSE_PA: {total_SENSE_PA:.2f}%")

Percentatge de total_SENSE_PA: 0.00%


#### Pvalor

##### Independència

In [411]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'] == 100.0, AMB_PA_MECVV['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_AMB_PA_MECVV:", p_value_12)

P-valor entre indepe_AMB_PA i indepe_AMB_PA_MECVV: 1.0


In [412]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'] == 100.0, SENSE_PA['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_SENSE_PA:", p_value_13)

P-valor entre indepe_AMB_PA i indepe_SENSE_PA: 1.0


In [413]:
stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Barthel_resultados'] == 100.0, SENSE_PA['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_AMB_PA_MECVV i SENSE_PA:", p_value_23)

P-valor entre indepe_AMB_PA_MECVV i SENSE_PA: 0.006056441109782507


##### Moderada

In [414]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(61, 95), AMB_PA_MECVV['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_AMB_PA i mod_AMB_PA_MECVV:", p_value_12)

P-valor entre mod_AMB_PA i mod_AMB_PA_MECVV: 0.5483429354628137


In [415]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(61, 95), SENSE_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre indepe_AMB_PA i mod_SENSE_PA:", p_value_13)

P-valor entre indepe_AMB_PA i mod_SENSE_PA: 0.05422099630917637


In [416]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'].between(61, 95), SENSE_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')  
print("P-valor entre mod_AMB_PA_MECVV i mod_SENSE_PA :", p_value_23)

P-valor entre mod_AMB_PA_MECVV i mod_SENSE_PA : 0.07458641569139825


##### Severa

In [417]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(21,60), AMB_PA_MECVV['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_AMB_PA_MECVV:", p_value_12)

P-valor entre sev_AMB_PA i sev_AMB_PA_MECVV: 0.4622499461255427


In [418]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(21,60), SENSE_PA['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_SENSE_PA:", p_value_13)

P-valor entre sev_AMB_PA i sev_SENSE_PA: 0.3463247915940385


In [419]:
t_stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'].between(21,60), SENSE_PA['Barthel_resultados'].between(21,60), alternative='two-sided' )
print("P-valor entre sev_AMB_PA_MECVV i sev_SENSE_PA:", p_value_23)

P-valor entre sev_AMB_PA_MECVV i sev_SENSE_PA: 0.16065967802771053


##### Total

In [420]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'] <= 20.0, AMB_PA_MECVV['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_AMB_PA_MECVV:", p_value_12)

P-valor entre total_AMB_PA i total_AMB_PA_MECVV: 0.31988618684011016


In [421]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'] <= 20.0, SENSE_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_SENSE_PA:", p_value_13)

P-valor entre total_AMB_PA i total_SENSE_PA: 0.01293706233933995


In [422]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'] <= 20.0, SENSE_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA_MECVV i total_SENSE_PA:", p_value_23)

P-valor entre total_AMB_PA_MECVV i total_SENSE_PA: 0.0033087017190950086


## Index del MNA

Primer convertiré la columna a tipus numèric ja que els valors que conté són de tipus objecte i amb aquest tipus no puc calcular la mitjana ni la desviació estàndard.

In [423]:
AMB_PA.loc[:, 'mna_resultats'] = pd.to_numeric(AMB_PA['mna_resultats'], errors='coerce')
AMB_PA_MECVV.loc[:, 'mna_resultats'] = pd.to_numeric(AMB_PA_MECVV['mna_resultats'], errors='coerce')
SENSE_PA.loc[:, 'mna_resultats'] = pd.to_numeric(SENSE_PA['mna_resultats'], errors='coerce')

### Mitjana

In [424]:
promedio_mna = AMB_PA['mna_resultats'].mean()
desviacion_estandar_mna = AMB_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [425]:
promedio_mna = AMB_PA_MECVV['mna_resultats'].mean()
desviacion_estandar_mna = AMB_PA_MECVV['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [426]:
promedio_mna = SENSE_PA['mna_resultats'].mean()
desviacion_estandar_mna = SENSE_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultado)

Mitjana ± Desviació Estàndard del mna: 0.00 ± 0.00


#### Pvalor

##### Normalitat

In [427]:
stat, p_value = shapiro(AMB_PA['mna_resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [428]:
stat, p_value = shapiro(AMB_PA_MECVV['mna_resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [429]:
stat, p_value = shapiro(SENSE_PA['mna_resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [430]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['mna_resultats'], AMB_PA_MECVV['mna_resultats'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [431]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['mna_resultats'], SENSE_PA['mna_resultats'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [432]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['mna_resultats'], SENSE_PA['mna_resultats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Valors de malnutrició

##### AMB_PA

In [433]:
norm_AMB_PA = AMB_PA['mna_resultats'].between(24, 30)
Percentatge_norm_AMB_PA = norm_AMB_PA.mean() * 100
num_observaciones_norm_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de norm_AMB_PA: {Percentatge_norm_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observaciones_norm_AMB_PA}")

Percentatge de norm_AMB_PA: 0.00%
Nombre d'observacions en norm_AMB_PA: 0


In [434]:
malnutr_AMB_PA = AMB_PA['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA = malnutr_AMB_PA.mean() * 100
num_observaciones_malnutr_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de malnutr_AMB_PA: {Percentatge_malnutr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observaciones_malnutr_AMB_PA}")

Percentatge de malnutr_AMB_PA: 0.00%
Nombre d'observacions en norm_AMB_PA: 0


In [435]:
malnut_AMB_PA = AMB_PA['mna_resultats'] < 17
Percentatge_malnut_AMB_PA = malnut_AMB_PA.mean() * 100
num_observaciones_malnut_AMB_PA = malnut_AMB_PA.sum()

print(f"Percentatge de malnut_AMB_PA: {Percentatge_malnut_AMB_PA:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA: {num_observaciones_malnut_AMB_PA}")

Percentatge de malnut_AMB_PA: 23.53%
Nombre d'observacions en malnut_AMB_PA: 4


##### AMB_PA_MECVV

In [436]:
norm_AMB_PA_MECVV = AMB_PA_MECVV['mna_resultats'].between(24, 30)
Percentatge_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.mean() * 100
num_observaciones_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de norm_AMB_PA_MECVV: {Percentatge_norm_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observaciones_norm_AMB_PA_MECVV}")

Percentatge de norm_AMB_PA_MECVV: 0.00%
Nombre d'observacions en norm_AMB_PA_MECVV: 0


In [437]:
malnutr_AMB_PA_MECVV = AMB_PA_MECVV['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA_MECVV = malnutr_AMB_PA_MECVV.mean() * 100
num_observaciones_malnutr_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de malnutr_AMB_PA_MECVV: {Percentatge_malnutr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observaciones_malnutr_AMB_PA_MECVV}")

Percentatge de malnutr_AMB_PA_MECVV: 0.00%
Nombre d'observacions en norm_AMB_PA_MECVV: 0


In [438]:
malnut_AMB_PA_MECVV = AMB_PA_MECVV['mna_resultats'] < 17
Percentatge_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.mean() * 100
num_observaciones_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.sum()

print(f"Percentatge de malnut_AMB_PA_MECVV: {Percentatge_malnut_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA_MECVV: {num_observaciones_malnut_AMB_PA_MECVV}")

Percentatge de malnut_AMB_PA_MECVV: 25.00%
Nombre d'observacions en malnut_AMB_PA_MECVV: 2


##### SENSE_PA

In [439]:
norm_SENSE_PA = SENSE_PA['mna_resultats'].between(24, 30)
Percentatge_norm_SENSE_PA = norm_SENSE_PA.mean() * 100
num_observaciones_norm_SENSE_PA = norm_SENSE_PA.sum()

print(f"Percentatge de norm_SENSE_PA: {Percentatge_norm_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en norm_SENSE_PA: {num_observaciones_norm_SENSE_PA}")

Percentatge de norm_SENSE_PA: 0.00%
Nombre d'observacions en norm_SENSE_PA: 0


In [440]:
malnutr_SENSE_PA = SENSE_PA['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_SENSE_PA = malnutr_SENSE_PA.mean() * 100
num_observaciones_malnutr_SENSE_PA = malnutr_SENSE_PA.sum()

print(f"Percentatge de malnutr_SENSE_PA: {Percentatge_malnutr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnutr_SENSE_PA: {num_observaciones_malnutr_SENSE_PA}")

Percentatge de malnutr_SENSE_PA: 0.00%
Nombre d'observacions en malnutr_SENSE_PA: 0


In [441]:
malnut_SENSE_PA = SENSE_PA['mna_resultats'] < 17
Percentatge_malnut_SENSE_PA = malnut_SENSE_PA.mean() * 100
num_observaciones_malnut_SENSE_PA = malnut_SENSE_PA.sum()

print(f"Percentatge de malnut_SENSE_PA: {Percentatge_malnut_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnut_SENSE_PA: {num_observaciones_malnut_SENSE_PA}")

Percentatge de malnut_SENSE_PA: 37.50%
Nombre d'observacions en malnut_SENSE_PA: 3


#### Pvalor

##### Normal

In [442]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['mna_resultats'].between(24, 30), AMB_PA_MECVV['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA i norm_AMB_PA_MECVV:", p_val_12)

P-valor entre norm_AMB_PA i norm_AMB_PA_MECVV: 1.0


In [443]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['mna_resultats'].between(24, 30), SENSE_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA i norm_SENSE_PA:", p_val_13)

P-valor entre norm_AMB_PA i norm_SENSE_PA: 1.0


In [444]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['mna_resultats'].between(24, 30), SENSE_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA_MECVV i norm_SENSE_PA:", p_val_23)

P-valor entre norm_AMB_PA_MECVV i norm_SENSE_PA: 1.0


##### Risc de malnutrició

In [445]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['mna_resultats'].between(17, 23.5), AMB_PA_MECVV['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA i malnutr_AMB_PA_MECVV:", p_val_12)

P-valor entre malnutr_AMB_PA i malnutr_AMB_PA_MECVV: 1.0


In [446]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['mna_resultats'].between(17, 23.5), SENSE_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA i malnutr_SENSE_PA:", p_val_13)

P-valor entre malnutr_AMB_PA i malnutr_SENSE_PA: 1.0


In [447]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['mna_resultats'].between(17, 23.5), SENSE_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA_MECVV i malnutr_SENSE_PA:", p_val_23)

P-valor entre malnutr_AMB_PA_MECVV i malnutr_SENSE_PA: 1.0


##### Malnutrició

In [448]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['mna_resultats'] <17 , 
                                AMB_PA_MECVV['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA i norm_AMB_PA_MECVV:", p_val_12)

P-valor entre malnut_AMB_PA i norm_AMB_PA_MECVV: 0.9686157491081862


In [449]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['mna_resultats'] <17 , 
                                SENSE_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA i malnut_SENSE_PA:", p_val_13)

P-valor entre malnut_AMB_PA i malnut_SENSE_PA: 0.5005446232443322


In [450]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['mna_resultats'] <17 , 
                                SENSE_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA_MECVV i malnut_SENSE_PA:", p_val_23)

P-valor entre malnut_AMB_PA_MECVV i malnut_SENSE_PA: 0.6477042192622259


## Index de l'EMINA

### Mitjana

In [451]:
promedio_emina = AMB_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = AMB_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 7.94 ± 3.21


In [452]:
promedio_emina = AMB_PA_MECVV['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = AMB_PA_MECVV['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 8.50 ± 1.51


In [453]:
promedio_emina = SENSE_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = SENSE_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 7.00 ± 2.83


#### Pvalor

##### Normalitat

In [454]:
stat, p_value = shapiro(AMB_PA['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9770
Valor p = 0.9248
El nombre de files segueix una distribució normal.


In [455]:
stat, p_value = shapiro(AMB_PA_MECVV['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9183
Valor p = 0.4158
El nombre de files segueix una distribució normal.


In [456]:
stat, p_value = shapiro(SENSE_PA['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8758
Valor p = 0.1716
El nombre de files segueix una distribució normal.


In [457]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'], AMB_PA_MECVV['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.5598635957447045


In [458]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'], SENSE_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.46879803068245907


In [459]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'], SENSE_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.21345611032992376


### Valors EMINA

##### AMB_PA

In [460]:
lowr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_AMB_PA = lowr_AMB_PA.mean() * 100
num_observaciones_lowr_AMB_PA = lowr_AMB_PA.sum()

print(f"Percentatge de lowr_AMB_PA: {Percentatge_lowr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA: {num_observaciones_lowr_AMB_PA}")

Percentatge de lowr_AMB_PA: 17.65%
Nombre d'observacions en lowr_AMB_PA: 3


In [461]:
modr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_AMB_PA = modr_AMB_PA.mean() * 100
num_observaciones_modr_AMB_PA = modr_AMB_PA.sum()

print(f"Percentatge de modr_AMB_PA: {Percentatge_modr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA: {num_observaciones_modr_AMB_PA}")

Percentatge de modr_AMB_PA: 64.71%
Nombre d'observacions en modr_AMB_PA: 11


In [462]:
highr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_AMB_PA = highr_AMB_PA.mean() * 100
num_observaciones_highr_AMB_PA = highr_AMB_PA.sum()

print(f"Percentatge de highr_AMB_PA: {Percentatge_highr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA: {num_observaciones_highr_AMB_PA}")

Percentatge de highr_AMB_PA: 17.65%
Nombre d'observacions en highr_AMB_PA: 3


##### AMB_PA_MECVV

In [463]:
lowr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.mean() * 100
num_observaciones_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.sum()

print(f"Percentatge de lowr_AMB_PA_MECVV: {Percentatge_lowr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA_MECVV: {num_observaciones_lowr_AMB_PA_MECVV}")

Percentatge de lowr_AMB_PA_MECVV: 0.00%
Nombre d'observacions en lowr_AMB_PA_MECVV: 0


In [464]:
modr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.mean() * 100
num_observaciones_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.sum()

print(f"Percentatge de modr_AMB_PA_MECVV: {Percentatge_modr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA_MECVV: {num_observaciones_modr_AMB_PA_MECVV}")

Percentatge de modr_AMB_PA_MECVV: 87.50%
Nombre d'observacions en modr_AMB_PA_MECVV: 7


In [465]:
highr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.mean() * 100
num_observaciones_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.sum()

print(f"Percentatge de highr_AMB_PA_MECVV: {Percentatge_highr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA_MECVV: {num_observaciones_highr_AMB_PA_MECVV}")

Percentatge de highr_AMB_PA_MECVV: 12.50%
Nombre d'observacions en highr_AMB_PA_MECVV: 1


##### SENSE_PA

In [466]:
lowr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_SENSE_PA = lowr_SENSE_PA.mean() * 100
num_observaciones_lowr_SENSE_PA = lowr_SENSE_PA.sum()

print(f"Percentatge de lowr_SENSE_PA: {Percentatge_lowr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en lowr_SENSE_PA: {num_observaciones_lowr_SENSE_PA}")

Percentatge de lowr_SENSE_PA: 37.50%
Nombre d'observacions en lowr_SENSE_PA: 3


In [467]:
modr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_SENSE_PA = modr_SENSE_PA.mean() * 100
num_observaciones_modr_SENSE_PA = modr_SENSE_PA.sum()

print(f"Percentatge de modr_SENSE_PA: {Percentatge_modr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modr_SENSE_PA: {num_observaciones_modr_SENSE_PA}")

Percentatge de modr_SENSE_PA: 62.50%
Nombre d'observacions en modr_SENSE_PA: 5


In [468]:
highr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_SENSE_PA = highr_SENSE_PA.mean() * 100
num_observaciones_highr_SENSE_PA = highr_SENSE_PA.sum()

print(f"Percentatge de highr_SENSE_PA: {Percentatge_highr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en highr_SENSE_PA: {num_observaciones_highr_SENSE_PA}")

Percentatge de highr_SENSE_PA: 0.00%
Nombre d'observacions en highr_SENSE_PA: 0


#### Pvalor 

##### Risc baix

In [469]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(0,5), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.08262239198015026


In [470]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(0,5), SENSE_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.35666312890880325


In [471]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), SENSE_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.07960201245519757


##### Risc moderat

In [472]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(6,10), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.20327534354395935


In [473]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(6,10), SENSE_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.9211148458207754


In [474]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), SENSE_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.2806702345204662


##### Risc alt

In [475]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(11,15), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7477825496543375


In [476]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(11,15), SENSE_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.08262239198015026


In [477]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), SENSE_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.3506166628202075


## Index de la canadenca 

### Mitjana

In [478]:
promedio_canad = AMB_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = AMB_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


In [479]:
promedio_canad = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: nan ± nan


In [480]:
promedio_canad = SENSE_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = SENSE_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


#### Pvalor

##### Normalitat

In [481]:
stat, p_value = shapiro(AMB_PA['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [482]:
stat, p_value = shapiro(AMB_PA_MECVV['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [483]:
stat, p_value = shapiro(SENSE_PA['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [484]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'], AMB_PA_MECVV['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [485]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'], SENSE_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [486]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'], SENSE_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Valors canadenca

##### AMB_PA 

In [487]:
mild_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_AMB_PA = mild_AMB_PA.mean() * 100
num_observaciones_mild_AMB_PA = mild_AMB_PA.sum()

print(f"Percentatge de mild_AMB_PA: {Percentatge_mild_AMB_PA:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA: {num_observaciones_mild_AMB_PA}")

Percentatge de mild_AMB_PA: 0.00%
Nombre d'observacions en mild_AMB_PA: 0


In [488]:
modneu_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_AMB_PA = modneu_AMB_PA.mean() * 100
num_observaciones_modneu_AMB_PA = modneu_AMB_PA.sum()

print(f"Percentatge de modneu_AMB_PA: {Percentatge_modneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA: {num_observaciones_modneu_AMB_PA}")

Percentatge de modneu_AMB_PA: 5.88%
Nombre d'observacions en modneu_AMB_PA: 1


In [489]:
sevneu_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA = sevneu_AMB_PA.mean() * 100
num_observaciones_sevneu_AMB_PA = sevneu_AMB_PA.sum()

print(f"Percentatge de sevneu_AMB_PA: {Percentatge_sevneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA: {num_observaciones_sevneu_AMB_PA}")

Percentatge de sevneu_AMB_PA: 0.00%
Nombre d'observacions en sevneu_AMB_PA: 0


##### AMB_PA_MECVV

In [490]:
mild_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.mean() * 100
num_observaciones_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.sum()

print(f"Percentatge de mild_AMB_PA_MECVV: {Percentatge_mild_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA_MECVV: {num_observaciones_mild_AMB_PA_MECVV}")

Percentatge de mild_AMB_PA_MECVV: 0.00%
Nombre d'observacions en mild_AMB_PA_MECVV: 0


In [491]:
modneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.mean() * 100
num_observaciones_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.sum()

print(f"Percentatge de modneu_AMB_PA_MECVV: {Percentatge_modneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA_MECVV: {num_observaciones_modneu_AMB_PA_MECVV}")

Percentatge de modneu_AMB_PA_MECVV: 0.00%
Nombre d'observacions en modneu_AMB_PA_MECVV: 0


In [492]:
sevneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.mean() * 100
num_observaciones_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.sum()

print(f"Percentatge de sevneu_AMB_PA_MECVV: {Percentatge_sevneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA_MECVV: {num_observaciones_sevneu_AMB_PA_MECVV}")

Percentatge de sevneu_AMB_PA_MECVV: 0.00%
Nombre d'observacions en sevneu_AMB_PA_MECVV: 0


##### SENSE_PA

In [493]:
mild_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_SENSE_PA = mild_SENSE_PA.mean() * 100
num_observaciones_mild_SENSE_PA = mild_SENSE_PA.sum()

print(f"Percentatge de mild_SENSE_PA: {Percentatge_mild_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en mild_SENSE_PA: {num_observaciones_mild_SENSE_PA}")

Percentatge de mild_SENSE_PA: 0.00%
Nombre d'observacions en mild_SENSE_PA: 0


In [494]:
modneu_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_SENSE_PA = modneu_SENSE_PA.mean() * 100
num_observaciones_modneu_SENSE_PA = modneu_SENSE_PA.sum()

print(f"Percentatge de modneu_SENSE_PA: {Percentatge_modneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modneu_SENSE_PA: {num_observaciones_modneu_SENSE_PA}")

Percentatge de modneu_SENSE_PA: 12.50%
Nombre d'observacions en modneu_SENSE_PA: 1


In [495]:
sevneu_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_SENSE_PA = sevneu_SENSE_PA.mean() * 100
num_observaciones_sevneu_SENSE_PA = sevneu_SENSE_PA.sum()

print(f"Percentatge de sevneu_SENSE_PA: {Percentatge_sevneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_SENSE_PA: {num_observaciones_sevneu_SENSE_PA}")

Percentatge de sevneu_SENSE_PA: 0.00%
Nombre d'observacions en sevneu_SENSE_PA: 0


#### Pvalor

##### Dèficit neurològic lleuger (mild)

In [496]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [497]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [498]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


##### Dèficit neurològic moderat

In [499]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'].between(5,10), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7798668588314288


In [500]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'].between(5,10), SENSE_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.3101221701595861


In [501]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10), SENSE_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.19078695285251068


##### Dèficit neurològic greu (sever)

In [502]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(1.5,4.5), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [503]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [504]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


## Infeccions respiratòries

##### AMB_PA 

In [505]:
mainresp_AMB_PA= (AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA:.2f}%")

Percentatge de infeccions respiratòries principals : 41.18%


In [506]:
LRTI_AMB_PA= (AMB_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA:.2f}%")

Percentatge de LRTI: 17.65%


In [507]:
COPD_AMB_PA= (AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA:.2f}%")

Percentatge de exacerbacions de COPD: 29.41%


In [508]:
fibro_AMB_PA= (AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


##### AMB_PA_MECVV

In [509]:
mainresp_AMB_PA_MECVV= (AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA_MECVV:.2f}%")

Percentatge de infeccions respiratòries principals : 50.00%


In [510]:
LRTI_AMB_PA_MECVV= (AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA_MECVV:.2f}%")

Percentatge de LRTI: 12.50%


In [511]:
COPD_AMB_PA_MECVV = (AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA_MECVV:.2f}%")

Percentatge de exacerbacions de COPD: 50.00%


In [512]:
fibro_AMB_PA_MECVV = (AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA_MECVV:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


##### SENSE_PA

In [513]:
mainresp_SENSE_PA= (SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_SENSE_PA:.2f}%")

Percentatge de infeccions respiratòries principals : 50.00%


In [514]:
LRTI_SENSE_PA= (SENSE_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_SENSE_PA:.2f}%")

Percentatge de LRTI: 25.00%


In [515]:
COPD_SENSE_PA= (SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_SENSE_PA:.2f}%")

Percentatge de exacerbacions de COPD: 25.00%


In [516]:
fibro_SENSE_PA= (SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_SENSE_PA:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


#### Pvalor

##### Normalitat

In [517]:
stat, p_value = shapiro(AMB_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6317
Valor p = 0.00002144
El nombre de files no segueix una distribució normal.


In [518]:
stat, p_value = shapiro(AMB_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4702
Valor p = 0.00000073
El nombre de files no segueix una distribució normal.


In [519]:
stat, p_value = shapiro(AMB_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5785
Valor p = 0.00000650
El nombre de files no segueix una distribució normal.


In [520]:
stat, p_value = shapiro(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [521]:
stat, p_value = shapiro(AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.00089170
El nombre de files no segueix una distribució normal.


In [522]:
stat, p_value = shapiro(AMB_PA_MECVV['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4184
Valor p = 0.00000105
El nombre de files no segueix una distribució normal.


In [523]:
stat, p_value = shapiro(AMB_PA_MECVV['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.00089170
El nombre de files no segueix una distribució normal.


In [524]:
stat, p_value = shapiro(AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


In [525]:
stat, p_value = shapiro(SENSE_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.00089170
El nombre de files no segueix una distribució normal.


In [526]:
stat, p_value = shapiro(SENSE_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5659
Valor p = 0.00006323
El nombre de files no segueix una distribució normal.


In [527]:
stat, p_value = shapiro(SENSE_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5659
Valor p = 0.00006323
El nombre de files no segueix una distribució normal.


In [528]:
stat, p_value = shapiro(SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


##### Infeccions respiratòries principals 

In [529]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0, AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7096213402025467


In [530]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0, SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.7096213402025467


In [531]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 1.0


##### LRTI

In [532]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['LRTI_diagnosticada'] == 1.0, AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7833089019485086


In [533]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['LRTI_diagnosticada'] == 1.0, SENSE_PA['LRTI_diagnosticada'] == 1.0,alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.7053744169042195


In [534]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0, SENSE_PA['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.587369453754309


##### Exacerbacions de COPD

In [535]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0, AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.34456236532569795


In [536]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0, SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.8515663144071834


In [537]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.34850142376108484


##### Fibrosi pulmonar i fibrotòrax

In [538]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 1.0


In [539]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 1.0


In [540]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 1.0


## Resultats de laboratori

Les columnes generades a partir d'aquests resultats contenen valors tipus objecte, per tant, primer els transformaré a tipus numèric per poder realitzar les operacions que neccessitaré fer.

### Albúmina

In [541]:
AMB_PA.loc[:, 'albumina'] = pd.to_numeric(AMB_PA['albumina'], errors='coerce')

promedio_alb = AMB_PA['albumina'].mean()
desviacion_estandar_alb = AMB_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.34 ± 0.38


In [542]:
AMB_PA_MECVV.loc[:, 'albumina'] = pd.to_numeric(AMB_PA_MECVV['albumina'], errors='coerce')

promedio_alb = AMB_PA_MECVV['albumina'].mean()
desviacion_estandar_alb = AMB_PA_MECVV['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.33 ± 0.38


In [543]:
SENSE_PA.loc[:, 'albumina'] = pd.to_numeric(SENSE_PA['albumina'], errors='coerce')

promedio_alb = SENSE_PA['albumina'].mean()
desviacion_estandar_alb = SENSE_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.55 ± 0.37


#### Pvalor

##### Normalitat

In [544]:
albpa = AMB_PA['albumina'].dropna()
stat, p_value = shapiro(albpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9311
Valor p = 0.56068160
El nombre de files segueix una distribució normal.


In [545]:
albAMBpamecvv = AMB_PA_MECVV['albumina'].dropna()
stat, p_value = shapiro(albAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8547
Valor p = 0.25296965
El nombre de files segueix una distribució normal.


In [546]:
albsensepa= SENSE_PA['albumina'].dropna()
stat, p_value = shapiro(albsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9840
Valor p = 0.92509128
El nombre de files segueix una distribució normal.


In [547]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['albumina'], AMB_PA_MECVV['albumina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [548]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['albumina'], SENSE_PA['albumina'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [549]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['albumina'], SENSE_PA['albumina'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Proteïnes totals

In [550]:
AMB_PA.loc[:, 'proteinas totales'] = pd.to_numeric(AMB_PA['proteinas totales'], errors='coerce')

promedio_prot = AMB_PA['proteinas totales'].mean()
desviacion_estandar_prot = AMB_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.03 ± 0.73


In [551]:
AMB_PA_MECVV.loc[:, 'proteinas totales'] = pd.to_numeric(AMB_PA_MECVV['proteinas totales'], errors='coerce')

promedio_prot = AMB_PA_MECVV['proteinas totales'].mean()
desviacion_estandar_prot = AMB_PA_MECVV['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.00 ± 0.53


In [552]:
SENSE_PA.loc[:, 'proteinas totales'] = pd.to_numeric(SENSE_PA['proteinas totales'], errors='coerce')

promedio_prot = SENSE_PA['proteinas totales'].mean()
desviacion_estandar_prot = SENSE_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.30 ± 0.80


#### Pvalor

##### Normalitat

In [553]:
protAMBpa=AMB_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9780
Valor p = 0.95346065
El nombre de files segueix una distribució normal.


In [554]:
protAMBpamecvv=AMB_PA_MECVV['proteinas totales'].dropna()
stat, p_value = shapiro(protAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8929
Valor p = 0.36311315
El nombre de files segueix una distribució normal.


In [555]:
protsensepa=SENSE_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9939
Valor p = 0.99129195
El nombre de files segueix una distribució normal.


In [556]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['proteinas totales'], AMB_PA_MECVV['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [557]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['proteinas totales'], SENSE_PA['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [558]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['proteinas totales'], SENSE_PA['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Hemoglobina

In [559]:
AMB_PA.loc[:, 'Hb'] = pd.to_numeric(AMB_PA['Hb'], errors='coerce')

promedio_hb = AMB_PA['Hb'].mean()
desviacion_estandar_hb = AMB_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 12.34 ± 1.77


In [560]:
AMB_PA_MECVV.loc[:, 'Hb'] = pd.to_numeric(AMB_PA_MECVV['Hb'], errors='coerce')

promedio_hb = AMB_PA_MECVV['Hb'].mean()
desviacion_estandar_hb = AMB_PA_MECVV['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 11.77 ± 2.58


In [561]:
SENSE_PA.loc[:, 'Hb'] = pd.to_numeric(SENSE_PA['Hb'], errors='coerce')

promedio_hb = SENSE_PA['Hb'].mean()
desviacion_estandar_hb = SENSE_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 12.83 ± 1.21


##### Pvalor

##### Normalitat

In [562]:
hbAMBpa=AMB_PA['Hb'].dropna()
stat, p_value = shapiro(hbAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9304
Valor p = 0.48501101
El nombre de files segueix una distribució normal.


In [563]:
hbAMBpamecvv=AMB_PA_MECVV['Hb'].dropna()
stat, p_value = shapiro(hbAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8288
Valor p = 0.18525796
El nombre de files segueix una distribució normal.


In [564]:
hbsensepa=SENSE_PA['Hb'].dropna()
stat, p_value = shapiro(hbsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9762
Valor p = 0.93094253
El nombre de files segueix una distribució normal.


In [565]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Hb'], AMB_PA_MECVV['Hb'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [566]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [567]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Colesterol 

In [568]:
AMB_PA.loc[:, 'colesterol total'] = pd.to_numeric(AMB_PA['colesterol total'], errors='coerce')

promedio_col = AMB_PA['colesterol total'].mean()
desviacion_estandar_col = AMB_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 132.10 ± 34.83


In [569]:
AMB_PA_MECVV.loc[:, 'colesterol total'] = pd.to_numeric(AMB_PA_MECVV['colesterol total'], errors='coerce')

promedio_col = AMB_PA_MECVV['colesterol total'].mean()
desviacion_estandar_col = AMB_PA_MECVV['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 151.47 ± 33.55


In [570]:
SENSE_PA.loc[:, 'colesterol total'] = pd.to_numeric(SENSE_PA['colesterol total'], errors='coerce')

promedio_col = SENSE_PA['colesterol total'].mean()
desviacion_estandar_col = SENSE_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 115.28 ± 23.69


#### Pvalor

##### Normalitat 



In [571]:
colAMBpa=AMB_PA['colesterol total'].dropna()
stat, p_value = shapiro(colAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9887
Valor p = 0.99048226
El nombre de files segueix una distribució normal.


In [572]:
colAMBpamecvv=AMB_PA_MECVV['colesterol total'].dropna()
stat, p_value = shapiro(colAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9941
Valor p = 0.85359605
El nombre de files segueix una distribució normal.


In [573]:
colsensepa=SENSE_PA['colesterol total'].dropna()
stat, p_value = shapiro(colsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9231
Valor p = 0.55007868
El nombre de files segueix una distribució normal.


In [574]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['colesterol total'], AMB_PA_MECVV['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [575]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['colesterol total'], SENSE_PA['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [576]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['colesterol total'], SENSE_PA['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Leucòcits

In [577]:
AMB_PA.loc[:, 'leucos'] = pd.to_numeric(AMB_PA['leucos'], errors='coerce')

promedio_leu = AMB_PA['leucos'].mean()
desviacion_estandar_leu = AMB_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 12.69 ± 4.88


In [578]:
AMB_PA_MECVV.loc[:, 'leucos'] = pd.to_numeric(AMB_PA_MECVV['leucos'], errors='coerce')

promedio_leu = AMB_PA_MECVV['leucos'].mean()
desviacion_estandar_leu = AMB_PA_MECVV['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 8.83 ± 0.70


In [579]:
SENSE_PA.loc[:, 'leucos'] = pd.to_numeric(SENSE_PA['leucos'], errors='coerce')

promedio_leu = SENSE_PA['leucos'].mean()
desviacion_estandar_leu = SENSE_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 10.68 ± 6.49


#### Pvalor 

##### Normalitat

In [580]:
leuambpa=AMB_PA['leucos'].dropna()
stat, p_value = shapiro(leuambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8482
Valor p = 0.09127190
El nombre de files segueix una distribució normal.


In [581]:
leuambpamecvv=AMB_PA_MECVV['leucos'].dropna()
stat, p_value = shapiro(leuambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8811
Valor p = 0.32754766
El nombre de files segueix una distribució normal.


In [582]:
leusensepa=SENSE_PA['leucos'].dropna()
stat, p_value = shapiro(leusensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8233
Valor p = 0.09427563
El nombre de files segueix una distribució normal.


In [583]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['leucos'], AMB_PA_MECVV['leucos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [584]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['leucos'], SENSE_PA['leucos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [585]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['leucos'], SENSE_PA['leucos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Limfòcits %

In [586]:
AMB_PA.loc[:, 'limfos'] = pd.to_numeric(AMB_PA['limfos'], errors='coerce')

promedio_limf = AMB_PA['limfos'].mean()
desviacion_estandar_limf = AMB_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 16.70 ± 12.29


In [587]:
AMB_PA_MECVV.loc[:, 'limfos'] = pd.to_numeric(AMB_PA_MECVV['limfos'], errors='coerce')

promedio_limf = AMB_PA_MECVV['limfos'].mean()
desviacion_estandar_limf = AMB_PA_MECVV['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 9.57 ± 2.10


In [588]:
SENSE_PA.loc[:, 'limfos'] = pd.to_numeric(SENSE_PA['limfos'], errors='coerce')

promedio_limf = SENSE_PA['limfos'].mean()
desviacion_estandar_limf = SENSE_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 26.66 ± 9.21


#### Pvalor

##### Normalitat

In [589]:
limfambpa=AMB_PA['limfos'].dropna()
stat, p_value = shapiro(limfambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8208
Valor p = 0.04757269
El nombre de files no segueix una distribució normal.


In [590]:
limfambpamecvv=AMB_PA_MECVV['limfos'].dropna()
stat, p_value = shapiro(limfambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9992
Valor p = 0.94750571
El nombre de files segueix una distribució normal.


In [591]:
limfsensepa=SENSE_PA['limfos'].dropna()
stat, p_value = shapiro(limfsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9815
Valor p = 0.94260450
El nombre de files segueix una distribució normal.


In [592]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['limfos'], AMB_PA_MECVV['limfos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [593]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['limfos'], SENSE_PA['limfos'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [594]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['limfos'], SENSE_PA['limfos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Proteïna C Reactiva

In [595]:
AMB_PA.loc[:, 'prot C react'] = pd.to_numeric(AMB_PA['prot C react'], errors='coerce')

promedio_protcr = AMB_PA['prot C react'].mean()
desviacion_estandar_protcr = AMB_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 8.90 ± 5.47


In [596]:
AMB_PA_MECVV.loc[:, 'prot C react'] = pd.to_numeric(AMB_PA_MECVV['prot C react'], errors='coerce')

promedio_protcr = AMB_PA_MECVV['prot C react'].mean()
desviacion_estandar_protcr = AMB_PA_MECVV['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 6.38 ± 3.51


In [597]:
SENSE_PA.loc[:, 'prot C react'] = pd.to_numeric(SENSE_PA['prot C react'], errors='coerce')

promedio_protcr = SENSE_PA['prot C react'].mean()
desviacion_estandar_protcr = SENSE_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 5.13 ± 5.30


#### Pvalor

##### Normalitat

In [598]:
pcrambpa=AMB_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9423
Valor p = 0.60665876
El nombre de files segueix una distribució normal.


In [599]:
pcrambpamecvv=AMB_PA_MECVV['prot C react'].dropna()
stat, p_value = shapiro(pcrambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8067
Valor p = 0.13058912
El nombre de files segueix una distribució normal.


In [600]:
pcrsensepa=SENSE_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8945
Valor p = 0.34256857
El nombre de files segueix una distribució normal.


In [601]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['prot C react'], AMB_PA_MECVV['prot C react'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [602]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['prot C react'], SENSE_PA['prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [603]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['prot C react'], SENSE_PA['prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Urea 

In [604]:
AMB_PA.loc[:, 'urea'] = pd.to_numeric(AMB_PA['urea'], errors='coerce')

promedio_urea = AMB_PA['urea'].mean()
desviacion_estandar_urea = AMB_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 62.17 ± 36.37


In [605]:
AMB_PA_MECVV.loc[:, 'urea'] = pd.to_numeric(AMB_PA_MECVV['urea'], errors='coerce')

promedio_urea = AMB_PA_MECVV['urea'].mean()
desviacion_estandar_urea = AMB_PA_MECVV['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 82.27 ± 32.15


In [606]:
SENSE_PA.loc[:, 'urea'] = pd.to_numeric(SENSE_PA['urea'], errors='coerce')

promedio_urea = SENSE_PA['urea'].mean()
desviacion_estandar_urea = SENSE_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 43.62 ± 18.84


#### Pvalor

##### Normalitat

In [607]:
uambpa=AMB_PA['urea'].dropna()
stat, p_value = shapiro(uambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8842
Valor p = 0.17374314
El nombre de files segueix una distribució normal.


In [608]:
uambpamecvv=AMB_PA_MECVV['urea'].dropna()
stat, p_value = shapiro(uambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9999
Valor p = 0.97770778
El nombre de files segueix una distribució normal.


In [609]:
usensepa=SENSE_PA['urea'].dropna()
stat, p_value = shapiro(usensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8322
Valor p = 0.11225495
El nombre de files segueix una distribució normal.


In [610]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['urea'], AMB_PA_MECVV['urea'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [611]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['urea'], SENSE_PA['urea'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [612]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['urea'], SENSE_PA['urea'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Filtrat glomerular CDK-EPI

In [613]:
AMB_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = AMB_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = AMB_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 56.00 ± 12.65


In [614]:
AMB_PA_MECVV.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA_MECVV['FGE CDK-EPI'], errors='coerce')

promedio_fg = AMB_PA_MECVV['FGE CDK-EPI'].mean()
desviacion_estandar_fg = AMB_PA_MECVV['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 48.00 ± 2.00


In [615]:
SENSE_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(SENSE_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = SENSE_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = SENSE_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 57.60 ± 16.27


#### Pvalor

##### Normalitat

In [616]:
fgepiambpa=AMB_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7690
Valor p = 0.01994941
El nombre de files no segueix una distribució normal.


In [617]:
fgepiambpamecvv=AMB_PA_MECVV['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


In [618]:
fgepisensepa=SENSE_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepisensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9605
Valor p = 0.81120317
El nombre de files segueix una distribució normal.


In [619]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], AMB_PA_MECVV['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [620]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [621]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Filtrat glomerular MDRD

In [622]:
AMB_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = AMB_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = AMB_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 51.50 ± 0.71


In [623]:
AMB_PA_MECVV.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA_MECVV['FGE MDRD'], errors='coerce')

promedio_fgmd = AMB_PA_MECVV['FGE MDRD'].mean()
desviacion_estandar_fgmd = AMB_PA_MECVV['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 51.00 ± nan


In [624]:
SENSE_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(SENSE_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = SENSE_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = SENSE_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 39.00 ± nan


#### Pvalor

##### Normalitat

In [625]:
fgmd = AMB_PA['FGE MDRD'].dropna()
stat, p_value = shapiro(fgmd)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")


ValueError: Data must be at least length 3.

In [626]:
stat, p_value = shapiro(AMB_PA_MECVV['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [627]:
stat, p_value = shapiro(SENSE_PA['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [628]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['FGE MDRD'], AMB_PA_MECVV['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [629]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [630]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i AMB_PA_MECVV:", p_val_23)

P-valor entre AMB_PA_MECVV i AMB_PA_MECVV: nan
